In [ ]:
# Text generation from n-gramms, based on a dataset

In [1]:
# utils
import numpy as np
import pandas as pd
import time
import nltk
import matplotlib.pyplot as plt
from textblob import TextBlob
import re
import itertools

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
import nltk

In [ ]:
nltk.download('wordnet')#Представляет собой электронный словарь-тезаурус и набор семантических сетей для английского языка.
nltk.download('stopwords')#  электронный словарь "пустых" слов
nltk.download('punkt')
# Лемматизация — это процесс преобразования слова в его базовую форму.
# токенизация - разбиение текста на слова ( и не-слова, те знаки препинания, границы абзацев и т.п) 

In [2]:
# код загрузки корпуса
import codecs
fileObj = codecs.open( "Great_Expectations.txt", "r", "utf_8_sig" )
raw_text = fileObj.read() # или читайте по строке
fileObj.close()

#### Make replace of numbers with numered word

In [3]:

ones = {
    0: '', 1: 'one', 2: 'two', 3: 'three', 4: 'four', 5: 'five', 6: 'six',
    7: 'seven', 8: 'eight', 9: 'nine', 10: 'ten', 11: 'eleven', 12: 'twelve',
    13: 'thirteen', 14: 'fourteen', 15: 'fifteen', 16: 'sixteen',
    17: 'seventeen', 18: 'eighteen', 19: 'nineteen'}
tens = {
    2: 'twenty', 3: 'thirty', 4: 'forty', 5: 'fifty', 6: 'sixty',
    7: 'seventy', 8: 'eighty', 9: 'ninety'}
illions = {
    1: 'thousand', 2: 'million', 3: 'billion', 4: 'trillion', 5: 'quadrillion',
    6: 'quintillion', 7: 'sextillion', 8: 'septillion', 9: 'octillion',
    10: 'nonillion', 11: 'decillion'}

def flatten_lists_in_lists(A):
    rt = []
    for i in A:
        if isinstance(i,list): rt.extend(i)
        else: rt.append(i)
    return rt


def num2word(input, tokenize = True):

    if str.isnumeric(input):
        input = int(input)
        if input < 0:
            return _join('negative', _num2word_pos(-input))
        if input == 0:
            return 'zero'
        result = _num2word_pos(input)


    else:
        result = input
    return result


def _num2word_pos(i):
    if i < 20:
        return ones[i]
    if i < 100:
        return _join(tens[i // 10], ones[i % 10])
    if i < 1000:
        return _divide(i, 100, 'hundred')
    for illions_number, illions_name in illions.items():
        if i < 1000**(illions_number + 1):
            break
    return _divide(i, 1000**illions_number, illions_name)


def _divide(dividend, divisor, magnitude):
    return _join(
        _num2word_pos(dividend // divisor),
        magnitude,
        _num2word_pos(dividend % divisor),
    )


def _join(*args):
    return ' '.join(filter(bool, args))

In [4]:
# testing
num2word(str('1354'))

'one thousand three hundred fifty four'

#### Tokenization

In [6]:
pattern = r'\w+'
tokenizer = RegexpTokenizer(pattern)
tokenized_text1 = tokenizer.tokenize(raw_text)

* С помощью nltk.word_tokenize.

In [7]:
tokenized_text2 = word_tokenize(raw_text)
# отчистим от знаков припинания
tokenized_text2 = [w for w in tokenized_text2 if w.isalpha()]

#### Lower and upper case transform

In [8]:
# lower case
tokenized_text2_l = [t.lower() for t in tokenized_text2]
tokenized_text2_l

['my',
 'fathers',
 'family',
 'name',
 'being',
 'pirrip',
 'and',
 'my',
 'christian',
 'name',
 'philip',
 'my',
 'infant',
 'tongue',
 'could',
 'make',
 'of',
 'both',
 'names',
 'nothing',
 'longer',
 'or',
 'more',
 'explicit',
 'than',
 'pip',
 'so',
 'i',
 'called',
 'myself',
 'pip',
 'and',
 'came',
 'to',
 'be',
 'called',
 'pip',
 'i',
 'give',
 'pirrip',
 'as',
 'my',
 'father',
 's',
 'family',
 'name',
 'on',
 'the',
 'authority',
 'of',
 'his',
 'tombstone',
 'and',
 'my',
 'sister',
 'joe',
 'gargery',
 'who',
 'married',
 'the',
 'blacksmith',
 'as',
 'i',
 'never',
 'saw',
 'my',
 'father',
 'or',
 'my',
 'mother',
 'and',
 'never',
 'saw',
 'any',
 'likeness',
 'of',
 'either',
 'of',
 'them',
 'for',
 'their',
 'days',
 'were',
 'long',
 'before',
 'the',
 'days',
 'of',
 'photographs',
 'my',
 'first',
 'fancies',
 'regarding',
 'what',
 'they',
 'were',
 'like',
 'were',
 'unreasonably',
 'derived',
 'from',
 'their',
 'tombstones',
 'the',
 'shape',
 'of',
 'th

In [9]:
# Upper case
tokenized_text2_u = [t.upper() for t in tokenized_text2]
tokenized_text2_u

['MY',
 'FATHERS',
 'FAMILY',
 'NAME',
 'BEING',
 'PIRRIP',
 'AND',
 'MY',
 'CHRISTIAN',
 'NAME',
 'PHILIP',
 'MY',
 'INFANT',
 'TONGUE',
 'COULD',
 'MAKE',
 'OF',
 'BOTH',
 'NAMES',
 'NOTHING',
 'LONGER',
 'OR',
 'MORE',
 'EXPLICIT',
 'THAN',
 'PIP',
 'SO',
 'I',
 'CALLED',
 'MYSELF',
 'PIP',
 'AND',
 'CAME',
 'TO',
 'BE',
 'CALLED',
 'PIP',
 'I',
 'GIVE',
 'PIRRIP',
 'AS',
 'MY',
 'FATHER',
 'S',
 'FAMILY',
 'NAME',
 'ON',
 'THE',
 'AUTHORITY',
 'OF',
 'HIS',
 'TOMBSTONE',
 'AND',
 'MY',
 'SISTER',
 'JOE',
 'GARGERY',
 'WHO',
 'MARRIED',
 'THE',
 'BLACKSMITH',
 'AS',
 'I',
 'NEVER',
 'SAW',
 'MY',
 'FATHER',
 'OR',
 'MY',
 'MOTHER',
 'AND',
 'NEVER',
 'SAW',
 'ANY',
 'LIKENESS',
 'OF',
 'EITHER',
 'OF',
 'THEM',
 'FOR',
 'THEIR',
 'DAYS',
 'WERE',
 'LONG',
 'BEFORE',
 'THE',
 'DAYS',
 'OF',
 'PHOTOGRAPHS',
 'MY',
 'FIRST',
 'FANCIES',
 'REGARDING',
 'WHAT',
 'THEY',
 'WERE',
 'LIKE',
 'WERE',
 'UNREASONABLY',
 'DERIVED',
 'FROM',
 'THEIR',
 'TOMBSTONES',
 'THE',
 'SHAPE',
 'OF',
 'TH

In [10]:
# apply num2word(t)
tokenized_text2_l_n = [num2word(t) for t in tokenized_text2_l]
# flatten lists
tokenized_text2_l_n = flatten_lists_in_lists(tokenized_text2_l_n)

#### Word frequency analysis

In [11]:
def custom_counter(word):
    counted = {}
    first_top = 10

    for letter in word:
        if letter not in counted:
            counted[letter] = 0
        counted[letter] += 1
    counted # has a type dict
    top_count = {k: v for k, v in sorted(counted.items(), key=lambda item: item[1], reverse=True)} # sort to top values in the dict
    return top_count

def save_dict_as_csv(in_dict,file_name, main_col_name ):
    df_from_dict = pd.DataFrame.from_dict(in_dict, columns=['Count'], orient='index')
    df_from_dict.index = df_from_dict.index.set_names([main_col_name])
    df_from_dict.to_csv(file_name)    

In [ ]:
top_count1 = custom_counter(tokenized_text2_l_n)
# save
save_dict_as_csv(top_count1,'word_counts_without_puncts.csv','Words' )
print("Sum counts of sumbols without puncts:",sum(top_count1.values()))
top_count1

#### n-gramm generation

In [13]:
bigrams_2_l_n = nltk.bigrams(tokenized_text2_l_n)
top_count2 = custom_counter(bigrams_2_l_n)
save_dict_as_csv(top_count2,'2_gramms_counts_without_puncts.csv','2-gramm' )
print("Sum counts of sumbols without puncts:",sum(top_count2.values()))
top_count2

Sum counts of sumbols without puncts: 185590


{('of', 'the'): 871,
 ('in', 'the'): 797,
 ('i', 'had'): 624,
 ('that', 'i'): 559,
 ('i', 'was'): 496,
 ('and', 'i'): 481,
 ('it', 'was'): 462,
 ('to', 'be'): 435,
 ('to', 'the'): 431,
 ('on', 'the'): 427,
 ('at', 'the'): 414,
 ('in', 'a'): 330,
 ('and', 'the'): 311,
 ('when', 'i'): 310,
 ('with', 'a'): 304,
 ('miss', 'havisham'): 299,
 ('that', 'he'): 284,
 ('don', 't'): 279,
 ('he', 'had'): 277,
 ('had', 'been'): 275,
 ('to', 'me'): 274,
 ('as', 'i'): 267,
 ('was', 'a'): 265,
 ('as', 'if'): 258,
 ('he', 'was'): 251,
 ('out', 'of'): 249,
 ('with', 'the'): 245,
 ('said', 'i'): 244,
 ('of', 'his'): 234,
 ('of', 'my'): 226,
 ('i', 'have'): 226,
 ('i', 'could'): 223,
 ('of', 'a'): 216,
 ('i', 'am'): 213,
 ('by', 'the'): 206,
 ('and', 'that'): 201,
 ('for', 'the'): 193,
 ('have', 'been'): 190,
 ('into', 'the'): 189,
 ('i', 'should'): 189,
 ('in', 'his'): 186,
 ('in', 'my'): 186,
 ('at', 'me'): 174,
 ('there', 'was'): 173,
 ('if', 'i'): 173,
 ('from', 'the'): 171,
 ('but', 'i'): 169,
 ('tha

* 3-грам

In [14]:
bigrams_3_l_n = nltk.trigrams(tokenized_text2_l_n)
top_count3 = custom_counter(bigrams_3_l_n)
save_dict_as_csv(top_count3,'3_gramms_counts_without_puncts.csv','3-gramm' )
print("Sum counts of sumbols without puncts:",sum(top_count3.values()))
top_count3

Sum counts of sumbols without puncts: 185589


{('i', 'don', 't'): 112,
 ('that', 'i', 'was'): 77,
 ('as', 'if', 'he'): 76,
 ('miss', 'havisham', 's'): 74,
 ('that', 'i', 'had'): 72,
 ('out', 'of', 'the'): 66,
 ('i', 'had', 'been'): 60,
 ('i', 'could', 'not'): 60,
 ('i', 'should', 'have'): 59,
 ('there', 'was', 'a'): 58,
 ('don', 't', 'know'): 57,
 ('it', 'was', 'a'): 56,
 ('that', 'it', 'was'): 55,
 ('that', 'he', 'was'): 52,
 ('if', 'i', 'had'): 49,
 ('don', 't', 'you'): 47,
 ('if', 'he', 'had'): 44,
 ('looked', 'at', 'me'): 41,
 ('when', 'i', 'was'): 40,
 ('i', 'was', 'not'): 40,
 ('when', 'i', 'had'): 39,
 ('as', 'if', 'it'): 38,
 ('that', 'he', 'had'): 37,
 ('to', 'be', 'a'): 36,
 ('that', 'i', 'could'): 36,
 ('that', 'i', 'should'): 36,
 ('if', 'he', 'were'): 35,
 ('and', 'i', 'had'): 35,
 ('it', 'was', 'not'): 35,
 ('and', 'that', 'i'): 34,
 ('would', 'have', 'been'): 33,
 ('i', 'felt', 'that'): 32,
 ('to', 'me', 'that'): 32,
 ('to', 'miss', 'havisham'): 32,
 ('as', 'if', 'i'): 31,
 ('was', 'going', 'to'): 31,
 ('at', 'the',

#### let's account puncts in the text

In [15]:
pattern = r'[\d.,]+|[A-Z][.A-Z]+\b\.*|\w+|\S'
tokenizer = RegexpTokenizer(pattern)
tokenized_text3 = tokenizer.tokenize(raw_text)
tokenized_text3

['My',
 'fathers',
 'family',
 'name',
 'being',
 'Pirrip',
 ',',
 'and',
 'my',
 'Christian',
 'name',
 'Philip',
 ',',
 'my',
 'infant',
 'tongue',
 'could',
 'make',
 'of',
 'both',
 'names',
 'nothing',
 'longer',
 'or',
 'more',
 'explicit',
 'than',
 'Pip',
 '.',
 'So',
 ',',
 'I',
 'called',
 'myself',
 'Pip',
 ',',
 'and',
 'came',
 'to',
 'be',
 'called',
 'Pip',
 '.',
 'I',
 'give',
 'Pirrip',
 'as',
 'my',
 'father',
 '’',
 's',
 'family',
 'name',
 ',',
 'on',
 'the',
 'authority',
 'of',
 'his',
 'tombstone',
 'and',
 'my',
 'sister',
 ',',
 '—',
 'Mrs',
 '.',
 'Joe',
 'Gargery',
 ',',
 'who',
 'married',
 'the',
 'blacksmith',
 '.',
 'As',
 'I',
 'never',
 'saw',
 'my',
 'father',
 'or',
 'my',
 'mother',
 ',',
 'and',
 'never',
 'saw',
 'any',
 'likeness',
 'of',
 'either',
 'of',
 'them',
 '(',
 'for',
 'their',
 'days',
 'were',
 'long',
 'before',
 'the',
 'days',
 'of',
 'photographs',
 ')',
 ',',
 'my',
 'first',
 'fancies',
 'regarding',
 'what',
 'they',
 'were',


* С помощью nltk.word_tokenize.

In [16]:
# учитываем еще и знаки
tokenized_text4 = word_tokenize(raw_text)
# нижний регистр
tokenized_text4_l_puncts = [t.lower() for t in tokenized_text4]
tokenized_text4_l_puncts

['my',
 'fathers',
 'family',
 'name',
 'being',
 'pirrip',
 ',',
 'and',
 'my',
 'christian',
 'name',
 'philip',
 ',',
 'my',
 'infant',
 'tongue',
 'could',
 'make',
 'of',
 'both',
 'names',
 'nothing',
 'longer',
 'or',
 'more',
 'explicit',
 'than',
 'pip',
 '.',
 'so',
 ',',
 'i',
 'called',
 'myself',
 'pip',
 ',',
 'and',
 'came',
 'to',
 'be',
 'called',
 'pip',
 '.',
 'i',
 'give',
 'pirrip',
 'as',
 'my',
 'father',
 '’',
 's',
 'family',
 'name',
 ',',
 'on',
 'the',
 'authority',
 'of',
 'his',
 'tombstone',
 'and',
 'my',
 'sister',
 ',',
 '—mrs',
 '.',
 'joe',
 'gargery',
 ',',
 'who',
 'married',
 'the',
 'blacksmith',
 '.',
 'as',
 'i',
 'never',
 'saw',
 'my',
 'father',
 'or',
 'my',
 'mother',
 ',',
 'and',
 'never',
 'saw',
 'any',
 'likeness',
 'of',
 'either',
 'of',
 'them',
 '(',
 'for',
 'their',
 'days',
 'were',
 'long',
 'before',
 'the',
 'days',
 'of',
 'photographs',
 ')',
 ',',
 'my',
 'first',
 'fancies',
 'regarding',
 'what',
 'they',
 'were',
 'lik

In [17]:
# применим преобразование цифр в слов num2word(t)
tokenized_text4_l_puncts_n = [num2word(t) for t in tokenized_text4_l_puncts]
# для того чтобы "развернуть" вложенные листы
tokenized_text4_l_puncts_n = flatten_lists_in_lists(tokenized_text4_l_puncts_n)

In [18]:
top_count1_wpuncts = custom_counter(tokenized_text4_l_puncts_n)
# save
save_dict_as_csv(top_count1_wpuncts,'word_counts_puncts.csv','Words' )
print("Sum counts of sumbols puncts:",sum(top_count1_wpuncts.values()))
top_count1_wpuncts

Sum counts of sumbols puncts: 228843


{',': 17185,
 'the': 8297,
 'and': 7020,
 '.': 6584,
 'i': 6547,
 'to': 5132,
 'of': 4538,
 'a': 4036,
 '“': 3945,
 '”': 3908,
 'in': 3047,
 'that': 3037,
 'was': 2811,
 'it': 2737,
 '’': 2595,
 'you': 2285,
 'he': 2237,
 'had': 2087,
 'my': 2055,
 'me': 1937,
 'his': 1848,
 'with': 1794,
 'as': 1756,
 'at': 1643,
 'on': 1419,
 'for': 1382,
 'said': 1347,
 ';': 1257,
 '?': 1216,
 's': 1158,
 'her': 1151,
 'him': 1112,
 'not': 1109,
 'have': 1083,
 'but': 1046,
 'be': 1045,
 '!': 986,
 'she': 910,
 'when': 870,
 'by': 814,
 'were': 789,
 'if': 789,
 'this': 786,
 'we': 778,
 'so': 777,
 'out': 774,
 'all': 742,
 'there': 730,
 'joe': 726,
 'mr.': 679,
 't': 670,
 'what': 661,
 'is': 661,
 'no': 638,
 'or': 623,
 'been': 607,
 'up': 594,
 'would': 594,
 'from': 586,
 'which': 520,
 'one': 501,
 'into': 492,
 'could': 479,
 'do': 456,
 'an': 448,
 'now': 436,
 'then': 434,
 'very': 395,
 'your': 393,
 'more': 387,
 'they': 379,
 'miss': 378,
 'know': 374,
 'come': 372,
 'little': 370,
 'u

In [19]:
bigrams_2_l_n = nltk.bigrams(tokenized_text4_l_puncts_n)
top_count2 = custom_counter(bigrams_2_l_n)
save_dict_as_csv(top_count2,'2_gramms_counts_puncts.csv','2-gramm' )
print("Sum counts of sumbols puncts:",sum(top_count2.values()))
top_count2

Sum counts of sumbols puncts: 228842


{(',', 'and'): 3725,
 ('.', '“'): 1362,
 ('”', '“'): 1249,
 ('’', 's'): 1157,
 (',', '”'): 1118,
 (',', 'i'): 1025,
 ('”', 'said'): 933,
 (',', '“'): 895,
 ('?', '”'): 892,
 ('of', 'the'): 865,
 ('in', 'the'): 795,
 ('.', 'i'): 734,
 ('’', 't'): 662,
 ('i', 'had'): 623,
 ('!', '”'): 586,
 ('that', 'i'): 536,
 ('i', 'was'): 493,
 ('and', 'i'): 476,
 ('“', 'i'): 467,
 ('it', 'was'): 456,
 (',', 'as'): 448,
 ('to', 'be'): 435,
 ('to', 'the'): 428,
 ('me', ','): 427,
 ('on', 'the'): 425,
 (',', 'that'): 423,
 ('at', 'the'): 413,
 (',', 'but'): 380,
 ('it', ','): 339,
 (',', 'with'): 338,
 ('”', 'i'): 338,
 ('in', 'a'): 329,
 ('when', 'i'): 310,
 ('and', 'the'): 307,
 ('with', 'a'): 304,
 ('miss', 'havisham'): 299,
 ('.', 'it'): 283,
 ('don', '’'): 282,
 ('.', 'the'): 280,
 (',', 'in'): 279,
 ('he', 'had'): 277,
 ('had', 'been'): 275,
 ('to', 'me'): 274,
 ('that', 'he'): 270,
 ('as', 'i'): 267,
 ('was', 'a'): 263,
 ('as', 'if'): 258,
 (',', 'the'): 252,
 ('he', 'was'): 251,
 ('out', 'of'): 

* 3-грам

In [20]:
bigrams_3_l_n = nltk.trigrams(tokenized_text4_l_puncts_n)
top_count3 = custom_counter(bigrams_3_l_n)
save_dict_as_csv(top_count3,'3_gramms_counts_puncts.csv','3-gramm' )
print("Sum counts of sumbols puncts:",sum(top_count3.values()))
top_count3

Sum counts of sumbols puncts: 228841


{(',', '”', 'said'): 682,
 ('?', '”', '“'): 442,
 (',', 'and', 'i'): 310,
 ('don', '’', 't'): 279,
 ('”', 'said', 'i'): 209,
 (',', 'and', 'the'): 182,
 (',', 'and', 'that'): 168,
 ('.', '“', 'i'): 163,
 ('.', 'it', 'was'): 150,
 ('”', '“', 'i'): 145,
 ('?', '”', 'said'): 132,
 (',', 'that', 'i'): 131,
 (',', 'and', 'he'): 126,
 (',', 'as', 'if'): 124,
 ('”', 'said', 'joe'): 121,
 ('it', '’', 's'): 119,
 ('!', '”', 'said'): 117,
 ('”', 'said', 'mr.'): 117,
 (',', '“', 'i'): 114,
 ('said', 'i', ','): 113,
 ('i', 'don', '’'): 111,
 ('me', ',', 'and'): 108,
 (',', 'with', 'a'): 101,
 ('!', '”', '“'): 101,
 ('”', 'said', 'he'): 99,
 (',', 'and', 'then'): 93,
 (',', '”', 'i'): 92,
 (',', 'and', 'had'): 92,
 ('it', ',', 'and'): 90,
 ('?', '”', 'i'): 90,
 ('said', 'i', '.'): 89,
 (',', 'pip', ','): 88,
 ('said', 'joe', ','): 84,
 ('to', 'me', ','): 83,
 ('i', '.', '“'): 82,
 ('i', '’', 'll'): 80,
 ('.', '“', 'you'): 76,
 ('as', 'if', 'he'): 76,
 ('that', 'i', 'was'): 76,
 (',', 'and', 'was'):

In [104]:
top_count2


{(',', 'and'): 3725,
 ('.', '“'): 1362,
 ('”', '“'): 1249,
 ('’', 's'): 1157,
 (',', '”'): 1118,
 (',', 'i'): 1025,
 ('”', 'said'): 933,
 (',', '“'): 895,
 ('?', '”'): 892,
 ('of', 'the'): 865,
 ('in', 'the'): 795,
 ('.', 'i'): 734,
 ('’', 't'): 662,
 ('i', 'had'): 623,
 ('!', '”'): 586,
 ('that', 'i'): 536,
 ('i', 'was'): 493,
 ('and', 'i'): 476,
 ('“', 'i'): 467,
 ('it', 'was'): 456,
 (',', 'as'): 448,
 ('to', 'be'): 435,
 ('to', 'the'): 428,
 ('me', ','): 427,
 ('on', 'the'): 425,
 (',', 'that'): 423,
 ('at', 'the'): 413,
 (',', 'but'): 380,
 ('it', ','): 339,
 (',', 'with'): 338,
 ('”', 'i'): 338,
 ('in', 'a'): 329,
 ('when', 'i'): 310,
 ('and', 'the'): 307,
 ('with', 'a'): 304,
 ('miss', 'havisham'): 299,
 ('.', 'it'): 283,
 ('don', '’'): 282,
 ('.', 'the'): 280,
 (',', 'in'): 279,
 ('he', 'had'): 277,
 ('had', 'been'): 275,
 ('to', 'me'): 274,
 ('that', 'he'): 270,
 ('as', 'i'): 267,
 ('was', 'a'): 263,
 ('as', 'if'): 258,
 (',', 'the'): 252,
 ('he', 'was'): 251,
 ('out', 'of'): 

Sentence generator from n-grams

In [141]:
import copy
import random

'''
Sentence generator from n-gramms
input dict format "(of', 'the'): 865, ..."
You can pre sort dict for better results
'''
#In this case, I have utilized bi-gramms and i have sorted them already

def build_sentence(work_dict:dict, max_deepth:int=3):
    list_of_ngramms = list()
    last_word = str()
    deepth = 0
    n = len(list(work_dict.keys())[0]) # compute order of n-gramm
    ngramms_dict_inv = {random.random():x for x,y in work_dict.items()} # inverse dict values and keys, with save original order
    print("Work dict size:", len(ngramms_dict_inv))
    builder_call(n, deepth, max_deepth, ngramms_dict_inv, last_word, list_of_ngramms)
    result_sentence = [val for sublist in list_of_ngramms for val in sublist ] # flatten list of lists
    result_sentence = ' '.join(result_sentence)
    print("Result:", result_sentence)
    

def builder_call(n, deepth, max_deepth, gramms_dict,last_word_p, list_of_ngramms):    
    last_word = copy.deepcopy(last_word_p)

    for k,v in gramms_dict.items():
        if deepth == max_deepth: 
            #print("Max depth achived")
            break
        if deepth == 0:
            #print("Start, depth = 1")
            last_word = v[n-1]
            #print("last word: ", last_word)
            element = v[0:(n-1)]
            list_of_ngramms.append(element)
            #print("added element:", element)
            deepth +=1
            builder_call(n, deepth,max_deepth, gramms_dict, last_word, list_of_ngramms)
            break
        elif last_word == v[0] and (random.randint(0, 3) == 0): # add a litle bias for generator variability
            #print("elif ok")    
            last_word = v[n-1]
            #print("last word: ", last_word)
            element = v[0:(n-1)]
            list_of_ngramms.append(element)
            #print("added element:", element)

            deepth +=1
            builder_call(n, deepth, max_deepth, gramms_dict, last_word, list_of_ngramms)
            break
        
        

    

In [153]:

build_sentence(top_count2, max_deepth=50)

Work dict size: 78738
Result: , and the old chap , and the man , and the man ’ and i have got home , ” the fire at a good fellow , the same reason for a few moments , ” i had a good , i could have you are you , but
